# Superoperator Method

This notebook implements and tests some of the superoperator formalism.

In [ ]:
from transmon_code.superoperators import *
from transmon_code.helpers import *
from qutip import *
from transmon_code.simulate import *
from transmon_code.transmon import Transmon
import numpy as np
import pickle

In [ ]:
# setup the transmon
tr = Transmon(n_levels=8, initial_state=0, Ω=2*np.pi*3981, α=2*np.pi*-199, dt=1/50000)

tr.X90_args = {'A': 0.9821623327698041, 'τ': 0.024, 'λ': 0, 'α': tr.α, 'ω': tr.Ω, 'δ':-5.070445236848315}

# set up the idealised PTM

PTM = PTMs(tr.n_levels)
X90_PTM_ideal = PTM.ideal_PTM("X90")

In [ ]:
# learn the X90 PTM

X90_PTM = PTM.learn_X90_PTM(tr)

In [ ]:
# save everything
with open("transmon_PTMs", "wb") as f:
    pickle.dump([tr, X90_PTM], f)

In [ ]:
# load everything
with open("transmon_PTMs", "rb") as f:
    tr, X90_PTM = pickle.load(f)

In [ ]:
# check that the PTM gives the same result as the pulse
%matplotlib qt

tr.ψ0 = make_state(tr.n_levels,"0")

target = make_state(tr.n_levels,"L")

res_sim, f = simulate(tr, tr.X90_args, target=target, plot=True)

res_PTM = PTM.apply_PTM(X90_PTM, tr.ψ0)
res_PTM_ideal = PTM.apply_PTM(X90_PTM_ideal, tr.ψ0)

In [ ]:
plot_bloch([res_sim[-1], res_PTM, res_PTM_ideal])

print(fidelity(res_sim[-1], res_PTM)**2)
print(fidelity(res_sim[-1], res_PTM_ideal)**2)
print(fidelity(res_PTM, res_PTM_ideal)**2)

In [ ]:
# check a single gate runs as expected

gate = "Y90"
g = U(gate_angles[gate])

# via simulation
print("Performing simulation...")
res_sim, _ = simulate_circuit(tr, g, plot=False)

# via ideal PTM
print("Calculating via PTMs...")
gate_PTM = PTM.ideal_PTM(gate)
res_PTM1 = PTM.apply_PTM(gate_PTM, tr.ψ0)

# via decomposition
circ_PTM = PTM.circuit_PTM(gate, X90_PTM)
res_PTMc = PTM.apply_PTM(circ_PTM, tr.ψ0)

plot_bloch([tr.ψ0, res_sim, res_PTM1, res_PTMc])

print(fidelity(res_sim, res_PTMc)**2)

In [ ]:
# check a circuit works as expected

circ = ["X90", "Z90", "Y90", "X180"]

# via pulse simulation
res_sim, _ = simulate_circuit(tr, circ, plot=False)

# via circuit decomposition
circ_PTM = PTM.circuit_PTM(circ, X90_PTM)
res_PTM = PTM.apply_PTM(circ_PTM, tr.ψ0)

plot_bloch(truncate([tr.ψ0, res_sim, res_PTM]))

In [ ]:
fidelity(res_sim, res_PTM)**2

In [ ]:
# calculate the error gate
error_gate = X90_PTM * PTM.ideal_PTM("X270")#.inv()
Qobj(np.eye(error_gate.dims[0][0])) - error_gate

In [ ]:
# perform RBT
lengths = [20]

f = PTM.RBT_PTM(tr, X90_PTM, lengths, 1)